The aim of this notebook will be to find the following data:
* decision tree to plot in the visualization
* the generated neighbourhood generated by LORE

Everything will be tested on the iris dataset for quick running times

### dataset import

In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import json
from sklearn.preprocessing import LabelEncoder

In [2]:
data_file = "iris.json"
# Load and prepare data
data = load_iris()
X = data.data
y = data.target
feature_names = list(data.feature_names)
target_names = list(data.target_names)

### LORE initial

read data

In [3]:
from lore_sa.dataset import TabularDataset
import pandas as pd

In [4]:
data_dict = {name: X[:, i] for i, name in enumerate(feature_names)}
target_name = 'target'
data_dict[target_name] = [target_names[i] for i in y]  # Map numerical targets to names

dataset = TabularDataset.from_dict(data_dict, 'target')
dataset.df.dropna(inplace = True)
dataset.df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


train model

In [5]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from lore_sa.bbox import sklearn_classifier_bbox

In [6]:

def train_model_generalized(dataset: TabularDataset, target_name:str, ):
    numeric_indices = [v['index'] for k, v in dataset.descriptor['numeric'].items()]
    categorical_indices = [v['index'] for k, v in dataset.descriptor['categorical'].items()]

    # Create preprocessor using dynamic indices
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_indices),
            ('cat', OrdinalEncoder(), categorical_indices)
        ]
    )

    # Remove rare classes with fewer than 2 instances
    valid_classes = dataset.df[target_name].value_counts()[dataset.df[target_name].value_counts() > 1].index
    dataset.df = dataset.df[dataset.df[target_name].isin(valid_classes)]

        # Select features and target
    X = dataset.df.iloc[:, numeric_indices + categorical_indices]  # Select all features
    y = dataset.df[target_name]

    # Split dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                test_size=0.3, random_state=42, stratify=y)

    model = make_pipeline(preprocessor, RandomForestClassifier(n_estimators=100, random_state=42))
    
    model.fit(X_train, y_train)
    
    return sklearn_classifier_bbox.sklearnBBox(model)

In [7]:
bbox = train_model_generalized(dataset, target_name)

In [8]:
dataset.descriptor

{'numeric': {'sepal length (cm)': {'index': 0,
   'min': 4.3,
   'max': 7.9,
   'mean': 5.843333333333334,
   'std': 0.828066127977863,
   'median': 5.8,
   'q1': 5.1,
   'q3': 6.4},
  'sepal width (cm)': {'index': 1,
   'min': 2.0,
   'max': 4.4,
   'mean': 3.0573333333333337,
   'std': 0.4358662849366982,
   'median': 3.0,
   'q1': 2.8,
   'q3': 3.3},
  'petal length (cm)': {'index': 2,
   'min': 1.0,
   'max': 6.9,
   'mean': 3.7580000000000005,
   'std': 1.7652982332594662,
   'median': 4.35,
   'q1': 1.6,
   'q3': 5.1},
  'petal width (cm)': {'index': 3,
   'min': 0.1,
   'max': 2.5,
   'mean': 1.1993333333333336,
   'std': 0.7622376689603465,
   'median': 1.3,
   'q1': 0.3,
   'q3': 1.8}},
 'categorical': {},
 'ordinal': {},
 'target': {'target': {'index': 4,
   'distinct_values': ['setosa', 'versicolor', 'virginica'],
   'count': {'setosa': 50, 'versicolor': 50, 'virginica': 50}}}}

### encoding decoding

In [9]:
from lore_sa.encoder_decoder import ColumnTransformerEnc

tabular_enc = ColumnTransformerEnc(dataset.descriptor)
ref_value = dataset.df.iloc[0].values[:-1]
encoded = tabular_enc.encode([ref_value])
decoded = tabular_enc.decode(encoded)

print(f"Original value: {ref_value}")
print(f"Encoded value: {encoded}")
print(f"Decoded value: {decoded}")

Original value: [5.1 3.5 1.4 0.2]
Encoded value: [[5.1 3.5 1.4 0.2]]
Decoded value: [[5.1 3.5 1.4 0.2]]


the encoder is created using the dataset.descriptor, which is a dict so it can be saved to json and the encoder is then created based on the content read from it

In [10]:
dataset.descriptor

{'numeric': {'sepal length (cm)': {'index': 0,
   'min': 4.3,
   'max': 7.9,
   'mean': 5.843333333333334,
   'std': 0.828066127977863,
   'median': 5.8,
   'q1': 5.1,
   'q3': 6.4},
  'sepal width (cm)': {'index': 1,
   'min': 2.0,
   'max': 4.4,
   'mean': 3.0573333333333337,
   'std': 0.4358662849366982,
   'median': 3.0,
   'q1': 2.8,
   'q3': 3.3},
  'petal length (cm)': {'index': 2,
   'min': 1.0,
   'max': 6.9,
   'mean': 3.7580000000000005,
   'std': 1.7652982332594662,
   'median': 4.35,
   'q1': 1.6,
   'q3': 5.1},
  'petal width (cm)': {'index': 3,
   'min': 0.1,
   'max': 2.5,
   'mean': 1.1993333333333336,
   'std': 0.7622376689603465,
   'median': 1.3,
   'q1': 0.3,
   'q3': 1.8}},
 'categorical': {},
 'ordinal': {},
 'target': {'target': {'index': 4,
   'distinct_values': ['setosa', 'versicolor', 'virginica'],
   'count': {'setosa': 50, 'versicolor': 50, 'virginica': 50}}}}

In [11]:
with open ("descriptor.json", "w") as f:
    json.dump(dataset.descriptor, f, indent=4)

the value to use for the prediction can be found via:

In [12]:
ref_value = dataset.df.iloc[0].values[:-1]
list(ref_value)

[5.1, 3.5, 1.4, 0.2]

and decoded using

In [13]:
encoded = tabular_enc.encode([ref_value])
decoded = tabular_enc.decode(encoded)
list(decoded[0])

[5.1, 3.5, 1.4, 0.2]

### neighborhood generation

In [14]:
from lore_sa.neighgen import RandomGenerator

select the istance to explain

In [15]:
num_row = 0
x = dataset.df.iloc[num_row][:-1]
x

sepal length (cm)    5.1
sepal width (cm)     3.5
petal length (cm)    1.4
petal width (cm)     0.2
Name: 0, dtype: object

encode it

In [16]:
z = tabular_enc.encode([x.values])[0] # remove the class feature from the input instance

creates the neighborhood

In [17]:
gen = RandomGenerator(bbox=bbox, dataset=dataset, encoder=tabular_enc, ocr=0.1)
neighbour = gen.generate(z, 100, dataset.descriptor, tabular_enc)
neighbour

array([[5.1, 3.5, 1.4, 1.3202377500938514],
       [5.1, 3.5, 1.4, 1.3202377500938514],
       [5.1, 3.5, 1.4, 1.5354793660377504],
       [5.1, 3.5, 1.4, 1.5354793660377504],
       [5.1, 3.5, 1.4, 1.5354793660377504],
       [6.460748070744338, 3.5, 1.7967164675583747, 1.5354793660377504],
       [6.460748070744338, 3.5, 1.7967164675583747, 1.5354793660377504],
       [6.460748070744338, 3.5, 1.7967164675583747, 1.5354793660377504],
       [6.460748070744338, 3.5, 1.7967164675583747, 1.5354793660377504],
       [6.460748070744338, 3.5, 1.7967164675583747, 1.5354793660377504],
       [6.460748070744338, 3.5, 1.7967164675583747, 1.5354793660377504],
       [6.460748070744338, 3.5, 1.7967164675583747, 1.5354793660377504],
       [6.460748070744338, 3.5, 1.7967164675583747, 1.5354793660377504],
       [6.460748070744338, 3.5, 1.7967164675583747, 1.5354793660377504],
       [6.460748070744338, 3.5, 1.7967164675583747, 2.0059886146876775],
       [6.460748070744338, 3.452216509401693, 1.79

### surrogate model

In [18]:
from lore_sa.surrogate import DecisionTreeSurrogate

create X, y and yz for the decision tree surrogate

In [19]:
# decode the neighborhood to be labeled by the blackbox model
neighb_train_X = tabular_enc.decode(neighbour)
neighb_train_y = bbox.predict(neighb_train_X)
# encode the target class to the surrogate model
neighb_train_yz = tabular_enc.encode_target_class(neighb_train_y.reshape(-1, 1)).squeeze()

train the surrogate model

In [20]:
dt = DecisionTreeSurrogate()
x = dt.train(neighbour, neighb_train_yz)

### data extraction for the plots

#### decision tree

get the decision tree _tree for plotting

In [21]:
dt.get_dt().tree_

In [22]:
from sklearn.tree import DecisionTreeClassifier
from dataclasses import dataclass
from typing import List, Optional
from dataclasses import asdict

@dataclass
class TreeNode:
    """Class to store decision tree node information"""
    # Unique identifier for the node in the tree
    node_id: int
    
    # The name of the feature used for the decision at this node. 
    # If the node is a leaf, this will be `None`.
    feature_name: Optional[str]
    
    # The threshold value for the feature used to split the data at this node. 
    # If the node is a leaf, this will be `None`.
    threshold: Optional[float]
    
    # The node ID of the left child node. If the node is a leaf, this will be `None`.
    left_child: Optional[int]
    
    # The node ID of the right child node. If the node is a leaf, this will be `None`.
    right_child: Optional[int]
    
    # Indicates whether this node is a leaf node (`True` if leaf, `False` if internal).
    is_leaf: bool
    
    # The class label predicted by the leaf node. 
    # Only set if the node is a leaf; otherwise, it is `None`.
    class_label: Optional[str]
    
    # The number of samples (data points) that reached this node during training.
    samples: int

def extract_tree_structure(tree_classifier: DecisionTreeClassifier, feature_names: List[str], target_names: List[str]) -> List[TreeNode]: 
    """
    Extract node information from a trained DecisionTreeClassifier

    Parameters:
    -----------
    tree_classifier : DecisionTreeClassifier
        A trained sklearn DecisionTreeClassifier
    feature_names : List[str]
        A list of feature names
    target_names : List[str]
        A list of target class labels

    Returns:
    --------
    List[TreeNode]
        List of TreeNode objects containing the tree structure
    """
    tree = tree_classifier.tree_
    
    nodes = []

    for node_id in range(tree.node_count):
        # Check if node is leaf
        is_leaf = tree.children_left[node_id] == -1

        # Get node information
        if is_leaf:
            # Get the class label based on the majority class in the leaf
            class_label_index = int(tree.value[node_id].argmax())
            class_label = target_names[class_label_index]
            
            node = TreeNode(
                node_id=node_id,
                feature_name=None,
                threshold=None,
                left_child=None,
                right_child=None,
                is_leaf=True,
                class_label=class_label,
                samples=int(tree.n_node_samples[node_id])
            )
        else:
            feature_name = feature_names[int(tree.feature[node_id])]
            threshold = float(tree.threshold[node_id])
            left_child = int(tree.children_left[node_id])
            right_child = int(tree.children_right[node_id])

            node = TreeNode(
                node_id=node_id,
                feature_name=feature_name,
                threshold=threshold,
                left_child=left_child,
                right_child=right_child,
                is_leaf=False,
                class_label=None,
                samples=int(tree.n_node_samples[node_id])
            )

        nodes.append(node)

    return nodes

def generate_decision_tree_visualization_data(nodes):
    """
    Save the tree structure to a JSON file
    
    Parameters:
    -----------
    nodes : List[TreeNode]
        List of TreeNode objects to save
    filename : str
        Path to save the JSON file
    indent : int
        Number of spaces for indentation
    """
    # Convert TreeNodes to dictionaries
    nodes_dict = [asdict(node) for node in nodes]
    
    return nodes_dict


In [23]:
with open ("loreTreeTestTabular.json", "w") as f:
    json.dump(generate_decision_tree_visualization_data (
        nodes= extract_tree_structure(
            tree_classifier=dt.get_dt(),
            feature_names=list(dataset.descriptor["numeric"].keys()),
            target_names=list(dataset.descriptor["target"]["target"]['distinct_values'])
        )
    ), f, indent=4)

#### PCA

In [24]:
dt.get_dt()

DecisionTreeClassifier()

In [25]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.spatial import Voronoi
from shapely.geometry import Polygon
from shapely.ops import unary_union
import networkx as nx
from sklearn.cluster import KMeans

def preprocess_data(X):
    """
    Standardize the data and apply PCA transformation.
    
    Parameters:
    -----------
    X : array-like
        Input features
        
    Returns:
    --------
    tuple
        (transformed_data, pca_model, scaler_model)
    """
    scaler = StandardScaler()
    pca = PCA(n_components=2)
    X_scaled = scaler.fit_transform(X)
    X_pca = pca.fit_transform(X_scaled)
    return X_pca, pca, scaler

def generate_decision_boundary_grid(X_pca, step=0.1):
    """
    Generate a grid for decision boundary visualization.
    
    Parameters:
    -----------
    X_pca : array-like
        PCA transformed features
    step : float, default=0.1
        Step size for the grid
        
    Returns:
    --------
    tuple
        (xx, yy) meshgrid arrays and (x_min, x_max, y_min, y_max) boundaries
    """
    x_min, x_max = X_pca[:, 0].min() - 1, X_pca[:, 0].max() + 1
    y_min, y_max = X_pca[:, 1].min() - 1, X_pca[:, 1].max() + 1
    xx, yy = np.meshgrid(
        np.arange(x_min, x_max, step),
        np.arange(y_min, y_max, step)
    )
    return xx, yy, (x_min, x_max, y_min, y_max)

def filter_points_by_class_kmeans(points, original_data, labels, threshold=500, threshold_multiplier=4, random_state=42):
    """
    Filter points using K-means clustering to reduce data density, while preserving the original order.
    
    Instead of stacking by class (which reorders the data), we collect the original indices for each class,
    perform the filtering, and then sort the indices to restore the original ordering.
        
    Parameters:
    -----------
    points : array-like
        Input points to filter
    original_data : array-like
        Original feature data corresponding to points
    labels : array-like
        Class labels for points
    threshold : int, default=500
        Maximum number of points per class
    threshold_multiplier : int, default=4
        Multiplier for initial sampling size
    random_state : int, default=42
        Random state for reproducibility
        
    Returns:
    --------
    tuple
        (filtered_points, filtered_original_data, filtered_labels)
    """
    selected_indices = []
    unique_classes = np.unique(labels)
    
    for cls in unique_classes:
        # Get the indices in the original order for this class.
        class_indices = np.where(labels == cls)[0]
        class_points = points[class_indices]
        n_points = len(class_points)
        
        if n_points > threshold:
            sample_size = min(threshold * threshold_multiplier, n_points)
            rng = np.random.RandomState(random_state)
            sampled_indices_local = rng.choice(n_points, size=sample_size, replace=False)
            sampled_points = class_points[sampled_indices_local]
            
            kmeans = KMeans(n_clusters=threshold, random_state=random_state, n_init=10)
            kmeans.fit(sampled_points)
            centroids = kmeans.cluster_centers_
            
            # For each centroid, choose the point closest to it
            for centroid in centroids:
                distances = np.linalg.norm(class_points - centroid, axis=1)
                closest_local_index = np.argmin(distances)
                # Map back to the original index
                selected_indices.append(class_indices[closest_local_index])
        else:
            selected_indices.extend(class_indices.tolist())
    
    # Sort the indices to maintain original order
    selected_indices = np.sort(selected_indices)
    
    filtered_points = points[selected_indices]
    filtered_original = original_data[selected_indices]
    filtered_labels = labels[selected_indices]
    return filtered_points, filtered_original, filtered_labels

def create_voronoi_regions(xx, yy, Z, class_names):
    """
    Create Voronoi regions for decision boundaries.
    
    Parameters:
    -----------
    xx : array-like
        X coordinates of the grid
    yy : array-like
        Y coordinates of the grid
    Z : array-like
        Predicted classes for the grid points
    class_names : list
        List of class names corresponding to numeric indices
        
    Returns:
    --------
    tuple
        (merged_regions, merged_classes) where merged_classes contains actual class names
    """
    # Identify unique class indices present in Z
    unique_z = np.unique(Z)
    
    # Create a mapping from original class indices to a new contiguous range starting from 0
    # because sometimes some of the classes are missing from the generated neighborhood
    index_map = {old_idx: new_idx for new_idx, old_idx in enumerate(sorted(unique_z))}
    
    # Apply the mapping to remap Z values
    # This ensures the class indices are normalized (starting from 0 and sequential)
    Z_remapped = np.array([index_map[z] for z in Z.ravel()]).reshape(Z.shape)
    
    # Initialize a graph for tracking adjacency of Voronoi regions
    G = nx.Graph()
    
    # Compute the Voronoi diagram from grid coordinates
    vor = Voronoi(np.c_[xx.ravel(), yy.ravel()])
    
    # Extract Voronoi regions and vertices
    regions, vertices = vor.regions, vor.vertices
    
    # Dictionaries to store region information
    region_class_map = {}  # Maps region index to class name
    region_polygons = []    # Stores the actual polygonal regions
    region_class_list = []  # Stores the class names corresponding to each region
    region_index_map = {}   # Maps region index to polygon index
    
    polygon_idx = 0  # Index tracker for polygon storage
    
    # Iterate through Voronoi regions and assign class labels
    for point_index, region_index in enumerate(vor.point_region):
        region = regions[region_index]
        
        # Check for valid region (i.e., does not contain infinite points)
        if not -1 in region and len(region) > 0:
            # Create a polygon for the region using its vertex indices
            polygon = Polygon([vertices[i] for i in region])
            region_polygons.append(polygon)
            
            # Retrieve the remapped class index from Z_remapped
            class_idx = int(Z_remapped.ravel()[point_index])
            
            # Ensure class index is within valid range
            if class_idx < 0 or class_idx >= len(class_names):
                raise ValueError(f"Invalid remapped class index {class_idx} for class_names {class_names}")
            
            # Map region index to class name
            region_class_map[region_index] = class_names[class_idx]
            region_class_list.append(class_names[class_idx])
            
            # Store mapping between Voronoi region index and our polygon index
            region_index_map[region_index] = polygon_idx
            
            # Add region to graph
            G.add_node(region_index)
            
            polygon_idx += 1
    
    # Identify adjacent regions with the same class and add edges to graph
    for (p1, p2), ridge_vertices in zip(vor.ridge_points, vor.ridge_vertices):
        if -1 in ridge_vertices:  # Ignore ridges extending to infinity
            continue
        
        r1, r2 = vor.point_region[p1], vor.point_region[p2]
        
        # Only connect regions if they belong to the same class
        if region_class_map.get(r1) == region_class_map.get(r2):
            G.add_edge(r1, r2)
    
    # Merge connected regions with the same class
    merged_regions = []
    merged_classes = []
    
    for component in nx.connected_components(G):
        # Merge all polygons in the connected component
        merged_polygon = unary_union([
            region_polygons[region_index_map[i]] 
            for i in component 
            if i in region_index_map
        ])
        merged_regions.append(merged_polygon)
        
        # Assign class based on the first region in the component
        merged_classes.append(region_class_map[list(component)[0]])
    
    return merged_regions, merged_classes

def format_pc_label(pc_loadings, feature_names, pc_index):
    """
    Format the principal component label with feature contributions.
    
    Parameters:
    -----------
    pc_loadings : array-like
        Principal component loadings
    feature_names : list
        List of feature names
    pc_index : int
        Index of the principal component
        
    Returns:
    --------
    str
        Formatted label
    """
    return f"PC{pc_index + 1}: " + ", ".join(
        [f"{name} ({value:+.2f})" for name, value in zip(feature_names, pc_loadings)]
    )

def generate_pca_visualization_data(feature_names, X, y, pretrained_tree, class_names, step=0.1):
    """
    Generate PCA visualization data and decision boundaries for a pre-trained decision tree.
    
    Parameters:
    -----------
    feature_names : list
        List of feature names
    X : array-like
        Input features
    y : array-like
        Target labels
    pretrained_tree : DecisionTreeClassifier
        Pre-trained decision tree classifier on original (non-PCA) data
    step : float, default=0.1
        Step size for decision boundary grid
        
    Returns:
    --------
    dict
        Visualization data including PCA coordinates, original data, and decision boundaries
    """
    # Transform data
    X_pca, pca, scaler = preprocess_data(X)
    
    # Generate grid in PCA space
    xx, yy, (x_min, x_max, y_min, y_max) = generate_decision_boundary_grid(X_pca, step)
    
    # Transform grid points back to original space for prediction
    grid_points = np.c_[xx.ravel(), yy.ravel()]
    grid_original = pca.inverse_transform(grid_points)
    grid_original = scaler.inverse_transform(grid_original)
    
    # Get predictions using the pre-trained tree
    Z = pretrained_tree.predict(grid_original).reshape(xx.shape)
    
    # Filter PCA points and original data
    filtered_pca_data, filtered_original_data, filtered_labels = filter_points_by_class_kmeans(
        X_pca, X, y, threshold=2000, threshold_multiplier=5
    )
    
    # Create Voronoi regions with class names
    merged_regions, merged_classes = create_voronoi_regions(xx, yy, Z, class_names)
    
    # Format PC labels
    pc1_label = format_pc_label(pca.components_[0], feature_names, 0)
    pc2_label = format_pc_label(pca.components_[1], feature_names, 1)
    
    # Convert original data to lists of pd.Series
    original_series_list = [
        pd.Series(row, index=feature_names).to_dict()
        for row in filtered_original_data
    ]
    
    # Prepare visualization data
    visualization_data = {
        "pcaData": filtered_pca_data.tolist(),
        "originalData": original_series_list,
        "targets": filtered_labels.tolist(),
        "decisionBoundary": {
            "regions": [list(p.exterior.coords) for p in merged_regions],
            "regionClasses": merged_classes,  # Now contains actual class names
            "xRange": [float(x_min), float(x_max)],
            "yRange": [float(y_min), float(y_max)],
        },
        "xAxisLabel": pc1_label,
        "yAxisLabel": pc2_label,
    }
    
    return visualization_data


In [26]:

with open ("lorePCATestTabular.json", "w") as f:
    json.dump(generate_pca_visualization_data (
        feature_names=feature_names, 
        class_names=dataset.descriptor["target"]["target"]['distinct_values'], 
        X=neighb_train_X, 
        y=neighb_train_y, 
        pretrained_tree=dt.get_dt(), 
        step = 0.1
    ), f, indent=4)